In [102]:
import numpy as np
import pandas as pd
import torch.nn as nn
import h5py
import pickle
import torch
import time
import yaml
import copy
import math
import traceback
import sys
import time
import os

import torch.nn.functional as F
import torchvision
import transformers
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader

file_dir = '/home/quang/Documents/XAI_env-main/data/processed/IMS/'
file_name = 'x_train.hdf5'

with h5py.File(file_dir + file_name, 'r') as file:
    dataset1 = file['x_train'][:]  # Replace 'dataset_name' with the actual dataset key or name
    dataset1_shape = dataset1.shape
    a = dataset1.dtype
print(dataset1_shape)

   

(7304, 20)


In [103]:
import h5py

file_dir = '/home/quang/Documents/XAI_env-main/data/processed/IMS/'
file_name = 'y_train.hdf5'

with h5py.File(file_dir + file_name, 'r') as file:
    dataset2 = file['y_train'][:]  # Replace 'dataset_name' with the actual dataset key or name
    dataset2_shape = dataset2.shape

print(dataset2_shape)

(7304, 3)


In [104]:
class CustomDataset:
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        current_sample = torch.tensor(self.data[idx, :], dtype=torch.float)
        current_target = torch.tensor(self.targets[idx], dtype=torch.float)
        data_shape = current_sample.shape[1:]
        return {
            "sample": current_sample,
            "target": current_target,
        }


In [105]:
data = dataset1
data.shape

(7304, 20)

In [106]:
targets = dataset2
targets.shape

(7304, 3)

In [107]:
custom_dataset1 = CustomDataset(data=data,targets=targets)

In [108]:
len(custom_dataset1)

7304

In [109]:
print(custom_dataset1)

In [110]:
custom_dataset1[0]

{'sample': tensor([0.1569, 0.0183, 0.0194, 0.0166, 0.1277, 0.0036, 0.0034, 0.0016, 0.0035,
         0.0034, 0.0028, 0.0036, 0.0047, 0.0036, 0.0046, 0.0066, 0.0068, 0.0120,
         0.0117, 0.0073]),
 'target': tensor([34.5307,  0.7729, 10.1465])}

In [111]:
train_loader = DataLoader(custom_dataset1,
                          batch_size=128,
                          shuffle=True)

In [112]:
for i,data in train_loader:
    print(i)

sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample


In [113]:
for i, data in enumerate(train_loader):
    print(i)
    print(data)

0
{'sample': tensor([[0.0779, 0.0216, 0.0234,  ..., 0.0127, 0.0102, 0.0060],
        [0.0815, 0.0129, 0.0226,  ..., 0.0089, 0.0123, 0.0072],
        [0.0704, 0.0160, 0.0264,  ..., 0.0081, 0.0074, 0.0032],
        ...,
        [0.0655, 0.0175, 0.0224,  ..., 0.0066, 0.0074, 0.0061],
        [0.0530, 0.0268, 0.0139,  ..., 0.0234, 0.0145, 0.0162],
        [0.0511, 0.0273, 0.0083,  ..., 0.0273, 0.0220, 0.0153]]), 'target': tensor([[2.2100e+01, 4.9466e-01, 2.2577e+01],
        [3.6871e+01, 8.2527e-01, 7.8062e+00],
        [2.4024e+01, 5.3772e-01, 2.0653e+01],
        [4.9653e+00, 7.2737e-01, 1.8611e+00],
        [3.4524e+01, 7.7274e-01, 1.0153e+01],
        [6.0312e+00, 1.3500e-01, 3.8646e+01],
        [1.7600e+01, 3.9394e-01, 2.7077e+01],
        [3.7774e+01, 8.4548e-01, 6.9034e+00],
        [4.2087e+01, 9.4202e-01, 2.5903e+00],
        [2.8704e+01, 6.4248e-01, 1.5973e+01],
        [4.3622e+01, 9.7637e-01, 1.0556e+00],
        [1.4450e+01, 3.2342e-01, 3.0227e+01],
        [3.3697e+01, 7.542

In [114]:
 class Autoencoder(nn.Module):
    def __init__(self, input_size,embed_dim, noise_level):
        super(Autoencoder, self).__init__()
        self.embed_dim = embed_dim
        self.input_size, self.hidden_dim, self.noise_level = input_size, embed_dim,noise_level
        self.fc1 = nn.Linear(self.input_size, self.hidden_dim)
        self.fc2 = nn.Linear(self.hidden_dim, self.input_size)
        
    def encoder(self,x):
        x = self.fc1(x)
        h1 = F.relu(x)
        return h1
    
    def mask(self,x):
        corrupted_x = x + self.noise_level + torch.randn_like(x)   # randn_like  Initializes a tensor where all the elements are sampled from a normal distribution.
        return corrupted_x
    
    def decoder(self, x):
        h2 = self.fc2(x)
        return h2
    
    def forward (self, x):
        out = self.mask(x) # Adding noise to feed the network
        encoder = self.encoder(out)
        decoder = self.decoder(encoder)
        return decoder 
 
 class TransformerAE(nn.Module):
    def __init__(self, input_sizet,noise_level,embed_dim, hidden_size, num_layers, num_heads, dropout):
        super(TransformerAE, self).__init__()

        self.embedding = nn.Linear(input_sizet, hidden_size)
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_size, nhead=num_heads),
            num_layers=num_layers
        )
        self.fc = nn.Linear(hidden_size, input_sizet)
        self.autoencoder = Autoencoder(input_size = input_sizet, noise_level = noise_level, embed_dim = embed_dim)

    def forward(self, input_sizet):
        decoder = self.autoencoder(input_sizet)
        embedded = self.embedding(decoder)
        embedded = embedded.permute(1, 0, 2)  # [sequence_len, batch_size, hidden_size]
        encoded = self.transformer_encoder(embedded)
        encoded = encoded.permute(1, 0, 2)  # [batch_size, sequence_len, hidden_size]
        decoded = self.fc(encoded)
        return decoded
    


In [115]:
model = TransformerAE(input_sizet = dataset1_shape[1],
            noise_level = 0.01,   
            embed_dim = 16,
            hidden_size = 64,
            num_layers = 2,
            num_heads = 4,
            dropout = 0.1)

In [116]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=2)
criterion = nn.MSELoss()

In [117]:
print(model)



TransformerAE(
  (embedding): Linear(in_features=20, out_features=64, bias=True)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (fc): Linear(in_features=64, out_features=20, bias=True)
  (autoencoder): Autoencoder(
    (fc1): Linear(in_features=20, out_features=16, bias=True)
    (fc2): Linear(in_features=16, out_features=20, bias

In [118]:
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for epoch in range(num_epochs):
    model.train()
    for i, batch in enumerate(train_loader):
        x = batch["sample"].to(device)
        rul = batch["target"].to(device)
        rul = torch.reshape(rul[:, 1], (-1, 1))
        #Forward
        x = x.unsqueeze(1)
        output = model(x)
        loss = criterion(output,rul)
        
        #Backward and Optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print("Epoch {}/{}. Iter {}/{}. Loss: {}".format(epoch+1, num_epochs, i+1, len(train_loader), loss))

Epoch 1/100. Iter 1/58. Loss: 0.6699637770652771
Epoch 1/100. Iter 2/58. Loss: 0.3776760995388031
Epoch 1/100. Iter 3/58. Loss: 0.24095216393470764
Epoch 1/100. Iter 4/58. Loss: 0.20846228301525116
Epoch 1/100. Iter 5/58. Loss: 0.18586088716983795
Epoch 1/100. Iter 6/58. Loss: 0.17363008856773376
Epoch 1/100. Iter 7/58. Loss: 0.16869451105594635
Epoch 1/100. Iter 8/58. Loss: 0.15127423405647278
Epoch 1/100. Iter 9/58. Loss: 0.15801730751991272
Epoch 1/100. Iter 10/58. Loss: 0.16030290722846985
Epoch 1/100. Iter 11/58. Loss: 0.15522249042987823
Epoch 1/100. Iter 12/58. Loss: 0.1562000811100006
Epoch 1/100. Iter 13/58. Loss: 0.16879495978355408
Epoch 1/100. Iter 14/58. Loss: 0.14679133892059326
Epoch 1/100. Iter 15/58. Loss: 0.1409251093864441
Epoch 1/100. Iter 16/58. Loss: 0.15442048013210297
Epoch 1/100. Iter 17/58. Loss: 0.15435679256916046
Epoch 1/100. Iter 18/58. Loss: 0.1388365477323532
Epoch 1/100. Iter 19/58. Loss: 0.13067486882209778
Epoch 1/100. Iter 20/58. Loss: 0.151100113987

KeyboardInterrupt: 